In [1]:
import pandas as pd
import os
import glob
from pandasql import sqldf


# Cargo archivos y creo referencias temporales

In [2]:
path = 'Resultados_T/*'
tipo_de_archivo = '*.csv'
archivos = glob.glob(path + tipo_de_archivo)
# Busca el archivo más reciente
#archivos
archivo_mas_reciente = max(archivos, key=os.path.getctime)
nombre_del_archivo = os.path.basename(archivo_mas_reciente)
nombre_del_archivo = nombre_del_archivo[:-4]

In [3]:
archivos

['Resultados_T\\MovilP_T-03-01-2024.csv',
 'Resultados_T\\MovilP_T-10-01-2024.csv',
 'Resultados_T\\MovilP_T-17.01-2024.csv',
 'Resultados_T\\MovilP_T-19-12-2023.csv',
 'Resultados_T\\MovilP_T-24-01-2024.csv',
 'Resultados_T\\MovilP_T-28-12-2023.csv',
 'Resultados_T\\MovilP_T-31-01-2024.csv']

## Cargo todas las tablas en el entorno 

In [4]:
# Ruta al directorio que contiene los archivos
directorio = 'Resultados_T'
# Dicc para almacenar los DataFrames
dfs = {}
# Itera sobre todos los archivos en el directorio
for filename in os.listdir(directorio):
    file_path = os.path.join(directorio, filename)
    # Verifica si el path es un archivo y no un directorio
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)  
        # Obtiene el nombre del archivo sin la extensión
        nombre_sin_extension = os.path.splitext(filename)[0]
        # Añade el DataFrame al diccionario
        dfs[nombre_sin_extension] = df
## Etiqueto cada Df por su temporalidad, añado una columna mas con dicha etiqueta         
for tiempo, df in dfs.items():
    df['TIME'] = tiempo

# Combina todos los dataframes en uno solo
df_combinado = pd.concat(dfs.values())
dfs #df convinado del diccionario.
## El resultado es un diccionario de df donde el nombre del df es su key y el df es el value 
df_combinado.drop(columns=["Speed Download FB (Compare to 1 week earlier %)", "User. Ter 4G(%)", "User. Ter 3G(%)", 
                           "User. Ter 2G(%)", "Transporte", "Round Trip Time FB (Difference From Best %)",
                           "Speed Download FB (Difference From Best %)", "Round Trip Time FB Rank", "Capacidad", "Round Trip Time FB (ms)",
                           "Speed Download FB (Mbps)",'Speed Download FB Rank'], inplace = True) # Usseles columns 
#df_combinado.info()

In [5]:
df_combinado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2767 entries, 0 to 396
Data columns (total 43 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Mejor Red                          1973 non-null   object 
 1   Mayor Competidor                   1973 non-null   object 
 2   Departamento                       2767 non-null   object 
 3   Provincia                          2767 non-null   object 
 4   Distrito                           2767 non-null   object 
 5   TD_2G                              2767 non-null   float64
 6   TD_3G                              2767 non-null   float64
 7   TD_4G                              2767 non-null   float64
 8   Users_Encendidos                   2767 non-null   float64
 9   Users_Voz                          2767 non-null   float64
 10  Users_Datos                        2767 non-null   float64
 11  DL_OSIP_Claro                      2629 non-null   float64
 12

# Extraigo solo las columnas que sera evaluadas como series temporales 

In [6]:
Df_4TM = df_combinado[['Departamento','Provincia','Distrito','TIME','Market Share Claro FB %','Users','PL_Claro','CEI_Malo']]
Df_4TM.TIME.unique()

array(['MovilP_T-03-01-2024', 'MovilP_T-10-01-2024',
       'MovilP_T-17.01-2024', 'MovilP_T-19-12-2023',
       'MovilP_T-24-01-2024', 'MovilP_T-28-12-2023',
       'MovilP_T-31-01-2024'], dtype=object)

In [7]:
"""Si la memoria no fuera un problema deberia exportar todos los df apilados,
    Como si lo es, tengo que hacer 2 archivos, 
    el Actual(archivo de scrap) y uno exclusivo con la columnas de la metricas temporales(Archivo Actual)."""

Df_4TM.to_csv('Consultas/MT.csv',index=False)

# Tools 

## Filtro Distritos por parametros
#En caso solo quiera detemrinados distrito


In [15]:
Df_Actual = dfs[nombre_del_archivo] #Market Share Claro FB %  Area Sin Cobertura
Df_D_Covertura_lowMS = Df_Actual[(Df_Actual['Market Share Claro FB %'] <= 0.25) & (Df_Actual['Market Share Claro FB %'] > 0) & 
          (Df_Actual['Area Sin Cobertura'] <= 30)]


In [16]:
List_of_DC = Df_D_Covertura_lowMS.Distrito.tolist()
len(List_of_DC)

28

## Creo diferencia de de usuaruos

In [2]:
# Ordena los archivos por fecha de creación
archivos_ordenados = sorted(archivos, key=os.path.getctime, reverse=True)

# Obtiene los nombres del archivo más reciente y del segundo más antiguo
nombres_de_archivos = [os.path.basename(archivo)[:-4] for archivo in archivos_ordenados[:1] + archivos_ordenados[-2:-1]]

# Devuelve una lista del archivo más reciente y del segundo más antiguo
nombres_de_archivos


NameError: name 'archivos' is not defined

In [34]:
base_df['DIFF_users'] =  dfs[nombres_de_archivos[0]]['Users'] - dfs[nombres_de_archivos[1]]['Users']

In [35]:
base_df

,Users_MovilP_T-19-12-2023,Market Share Claro FB %,Area Sin Cobertura,DL_OSIP_Claro,Distrito,Provincia,Departamento,Users_MovilP_T-03-01-2024,Market Share Claro FB %_MovilP_T-03-01-2024,Area Sin Cobertura_MovilP_T-03-01-2024,DL_OSIP_Claro_MovilP_T-03-01-2024,Users_MovilP_T-28-12-2023,Market Share Claro FB %_MovilP_T-28-12-2023,Area Sin Cobertura_MovilP_T-28-12-2023,DL_OSIP_Claro_MovilP_T-28-12-2023,DIFF_users
0,4463,0.4753,7.0,8.58,IMAZA,BAGUA,AMAZONAS,4529,0.4721,7.0,7.72,4500,0.4506,7.0,7.72,29.0
1,1086,0.0774,100.0,0.00,NIEVA,CONDORCANQUI,AMAZONAS,1047,0.0756,100.0,0.00,1071,0.0737,100.0,0.00,-24.0
2,4760,0.4238,38.0,13.14,CAJARURO,UTCUBAMBA,AMAZONAS,5116,0.4345,38.0,12.97,4644,0.4279,38.0,12.97,472.0
3,14581,0.3553,19.0,13.72,BAGUA GRANDE,UTCUBAMBA,AMAZONAS,15111,0.3615,19.0,12.88,14497,0.3552,19.0,12.88,614.0
4,2512,0.4783,24.0,0.00,JAMALCA,UTCUBAMBA,AMAZONAS,2577,0.0000,24.0,18.03,2552,0.4667,24.0,18.03,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,2182,0.7976,7.0,11.69,MATAPALO,ZARUMILLA,TUMBES,2187,0.7475,7.0,11.48,2137,0.8831,7.0,11.48,3392.0
386,3645,0.3503,2.0,11.77,AGUAS VERDES,ZARUMILLA,TUMBES,3565,0.3592,2.0,13.64,3629,0.3556,2.0,13.64,117.0
387,32106,0.3625,2.0,13.42,TUMBES,TUMBES,TUMBES,32626,0.3659,2.0,12.55,32271,0.3661,2.0,12.55,1054.0
388,6918,0.3329,2.0,12.23,ZARUMILLA,ZARUMILLA,TUMBES,7215,0.3397,2.0,11.36,6891,0.3370,2.0,11.36,1180.0
